# Precessamento dos dados 

Após importar os dados, chegou a hora de verificarmos se estão adequados para análise.
As tarefas são as seguintes:

- [x] Estudar o dicionário de dados (arquivo METODOLOGIA.docx) e marcar o que for importante 
- [ ] Verificar o tipo dos dados
- [ ] Verificar a integridade dos dados:
  - [ ] Todas as colunas correspondem ao descrito no dicionário de dados?
  - [ ] Há duplicatas?
  - [ ] Há dados ausentes? São significativos?
  - [ ] Há dados incorretos, mal formatados, etc.?

[Link da documentação para trabalhar a consistencia das datas](https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html)

## Conferindo a integridade dos dados

Na etapa anterior, [Coleta e Preparação dos Dados]("./03_coleta_e_preparação_dos_dados.ipynb"), utilizamos o seguinte script para importar os dados e transformar o nome dos arquivos em um padrão adequado: 

In [1]:
import os
import re
import shutil
import pandas as pd

input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".xls"):
            match = pattern.search(filename)
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename)

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_21263/902107909.py:49: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


Observe que, ao realizar a importação, recebemos um aviso `DtypeWarning`. Iremos começar verificar a consistência e integridade dos dados compreendendo a razão pela qual este aviso ocorre e checar como podemos processar os dados adequadamente para torná-los úteis para análise. 

### Solucionando os problemas de tipo dos dados

A saída `DtypeWarning` do comando de importação nos avisa que o `pandas` não conseguiu identificar o tipo de algumas colunas do dataset e, por isso, teve que inferir um tipo. Em outras palavras, o `pandas` tenta determinar os tipos de cada coluna; uma operação que exige muita memória.  

Podemos silenciar esses avisos basicamente de duas formas: A primeira é utilizando o parâmetro `dtypes` e passando `object` como valor.`Object` [é a classe base de onde derivam toda as classes em Python](https://docs.python.org/pt-br/3/library/functions.html?highlight=object#object), ou seja, `object` pode conter qualquer objeto Python. Isso fará com que o `pandas` defina o tipo `object` para todos os dados. Esse tipo já é aplicado para tipos de dados mistos ou que o `pandas` não consegue inferir. [No entanto, essa medida deve ser evitado na medida do possível](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) (para desempenho e interoperabilidade com outras bibliotecas e métodos).

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                dtypes=object) 
                for file in file_path_genarator]
```

A outra forma é usar o parâmetro `low-memory=False`, como sugere o próprio aviso: `DtypeWarning: Columns (...,...) have mixed types. Specify dtype option on import or set low_memory=False`.

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                low-memory=True) 
                for file in file_path_genarator]
```
 Porém, esse parâmetro, embora aparente funcionar, é considerado obsoleto e pode levar a erros silenciosos que não são documentados. [Existe uma discussão que se arrasta desde 2014 sobre se este parâmetro deve ou não ser melhor documentado ou retirado da biblioteca](https://github.com/pandas-dev/pandas/issues/5888). O fato é que sua utilização não é muito comum apesar do aviso indicar seu uso.

De toda forma, nenhuma das abordagens resolvem o problema de fato. O `pandas` continuará usando muito recurso para adivinhar os tipos. 

Portanto, a abordagem mais indicada nesses casos é fornecer os tipos manualmente. 

Antes de partir para esta tarefa, vamos checar o estado do DataFrame:


In [2]:
df.shape

(119158, 54)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o

A saída desse método `info()` nos fornece as seguintes informações:

Column: Esta coluna indica os nomes das colunas presentes no DataFrame.

Non-Null Count: Esta coluna mostra a contagem de valores não nulos em cada coluna. Em outras palavras, é o número de entradas que possuem valores válidos para aquela coluna. Por exemplo, na primeira coluna "ANO_BO", existem 119158 valores não nulos.

Dtype: Esta coluna informa o tipo de dados (data type) presente em cada coluna. Por exemplo, `object`, como vimos, indica que os registros podem ser de qualquer tipo, geralmente são strings ou valores mistos. Manter os dados assim pode causar inúmeros problemas já que strings e não-strings podem estar juntas em uma mesma série.

Podemos ver que, com exceção de alguns tipos `float64`, o pandas inferiu `object` em todo o conjunto de dados. Trabalhar com os tipos corretos no DataFrame é extremamente importante para que possamos garantir:
- Precisão nos dados
- Economia de memória
- Desempenho
- Manipulação dos dados mais eficaz
- Prevenção de erros
- Consistência. 

Vamos olhar uma amostra aleatória dos dados para termos uma noção da tipagem que precisará ser adequada.

In [6]:
df.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
43625,2023,3517,3517/2023,13/06/2023 17:31:02,13/06/2023 17:31:02,13/06/2023,06:17,PELA MANHÃ,13/06/2023,13/06/2023 17:31:02,Desconhecida,Não,NaN,AVENIDA ELDORADO,10.0,JD DA ESTAÇÃO,ITAQUAQUECETUBA,SP,"-23,479395214","-46,3667920569999",Via pública,NaN,APRECIAÇÃO DO DELEGADO TITULAR,DEL.POL.ITAQUAQUECETUBA,DEL.POL.ITAQUAQUECETUBA,Localização e/ou Devolução,Localização/Apreensão e Entrega de veículo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RNR0F11,MG,BELO HORIZONTE,Branco,VW/T CROSS HL TSI AE,2021.0,2021.0,AUTOMOVEL,1.0,Apple
95209,2023,1314,1314/2023,02/03/2023 09:42:55,02/03/2023 09:42:55,27/02/2023,20:40,A NOITE,02/03/2023,02/03/2023 09:42:55,Desconhecida,Não,NaN,Rua Gonçalo Francisco Xavier,41.0,Jardim Cidade Pirituba,S.PAULO,SP,"-23,4771828","-46,7194106",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,33º D.P. PIRITUBA,87º D.P. V. P. BARRETO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEY2C20,SP,S.PAULO,Prata,I/VOLVO XC60 3.0T AWD,2009.0,2010.0,CAMIONETA,1.0,Apple
87119,2023,615836,615836/2023,22/04/2023 09:42:39,22/04/2023 09:42:44,19/04/2023,22:00,A NOITE,20/04/2023,22/04/2023 09:42:39,Desconhecida,Não,NaN,Rua Joaquim Antunes,501.0,Pinheiros,S.PAULO,SP,"-23,5643745","-46,681808",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,14º D.P. PINHEIROS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
111608,2023,383,383/2023,22/03/2023 18:52:44,22/03/2023 18:52:44,22/03/2023,18:10,A NOITE,22/03/2023,22/03/2023 18:52:44,Desconhecida,Não,NaN,RUA DARIO FERREIRA MARTINS,100.0,VILA CURUCA,S.PAULO,SP,"-23,4924346719999","-46,429876796",Via pública,NaN,BO PARA INVESTIGAÇÃO,22º D.P. SAO MIGUEL PTA,22º D.P. SAO MIGUEL PTA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Motorola
75809,2023,98165,98165/2023,04/04/2023 23:48:56,04/04/2023 23:49:14,03/04/2023,NaN,A NOITE,04/04/2023,04/04/2023 23:48:56,Desconhecida,Não,NaN,RUA MANOEL FERREIRA PIRES,320.0,VILA FORMOSA,S.PAULO,SP,"-23,581870592","-46,5266242289999",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 1,41º D.P. VILA RICA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple


Para fornecer ao `pandas` as informações corretas sobre os tipos dos dados, precisamos ter certeza de quais tipos eles são. Nesse ponto o dicionário de dados é fundamental . Como vimos no início da etapa anterior ([Coleta e Preparação dos Dados]("./03_coleta_e_preparação_dos_dados.ipynb")) A Secretaria de Segurança fornece um arquivo de texto intitulado "METODOLOGIA" contendo informações sobre o conjunto de dados. 



#### Conferindo o dicionário de dados 

Como uma tarefa complementar ao nosso objetivo nesta etapa, para facilitar a comparação entre o dicionário e o DataFrame, eu exportei a saída da propriedade `dtypes` e copiei a tabela do documento "METODOLOGIA" para uma planilha a fim de:

1. Checar se o DataFrame contém todas as variáveis presentes no dicionário de dados;
2. Identificar os tipos dos dados e atribuí-los adequadamente ao DataFrame.

A planilha se encontra no diretório `data` e, como poderá ser observado, fiz uma busca vertical (VLOOKUP ou PROCV) para cruzar as colunas que se referem às características do conjunto de dados. O código da exportação dos dtypes segue abaixo:

In [8]:
t = df.dtypes.reset_index()
t.columns = ["Variáveis", "Tipos"]
t.to_excel("./data/dtypes_out.xlsx", index=False)

Ao analisar o dicionário de dados, cheguei à conclusão de que ele oferece pouca ajuda para compreender o significado das variáveis além do que é possível inferir em uma comparação direta óbvia. Os motivos são destacados a seguir:

1. O dicionário não informa os tipos dos dados;
2. As descrições das variáveis são vagas e não fornecem mais informações além da repetição do nome da variável;
3. O nome das variáveis no dicionário são diferentes dos nomes no DataFrame, mas algumas se referem à mesma coisa. Por exemplo: `BOLETIM_EMITIDO` não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO".
4. O DataFrame possui 54 variáveis enquanto o dicionário indica 62. Dada a condição acima, não consigo afirmar se o conjunto de dados possui variáveis não especificadas no dicionário de dados ou se as variáveis do dicionário se referem exclusivamente às do conjunto de dados.

Sobre os registros do conjunto:

1. O dicionário de dados informa que, várias linhas podem se referir ao mesmo boletim. Portanto, segundo o documento, para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades das seguintes variáveis:

- NOME_DELEGACIA
- ANO_BO
- NUM_BO

Isso será checado posteriormente.

2. Existe inconsistência nos registros: alguns deles estão com caracteres maiúsculos e outros minúsculos.
3. Os registros de `LATITUDE` e `LONGITUDE` possuem um espaço em branco tanto à esquerda quanto à direita. Será necessário limpar estes espaços, substituir a "," por "." e converter o dados adequadamente para `float`.
4. A única coluna que se refere a valores do tipo `int` e que pode ser utilizada para cálculo é "QUANT_CELULAR", que se refere a quantidade de dispositivos roubados da vítima. Esta variável precisa ser convertida para o tipo adequado que suporte valores ausentes.
5. Existem muitos valores ausentes. Será necessário pensar em uma abordagem para lidar com eles.




#### Corrigindo os tipos e limpando os dados

Para formatar corretamente os registros de Longitude e Latitude usaremos a seguinte função:

In [ ]:

def clean_coordinate_value(value):
    if isinstance(value, str):
        try:
            clean_value = value.strip().replace(",", ".")
            return float(clean_value)
        except ValueError:
            return None 
        
    return None

O método `read_csv()` possui diversos argumentos nomeados que podemos fornecer para cuidar da formatação dos registros. Estes argumentos precisam ser estruturas de dados específicas:
- `dtype`: Precisa ser um tipo, como `str`, ou um dicionário. Esse argumento nomeado define os tipos de dados a serem aplicados a todo o conjunto de dados ou a colunas individuais
- `parse_dates`: `bool`, lista de Hashable, lista de listas ou dict de {Hashable list}, padrão `False`.  
- `date_format`: `str` ou `dict`  com nomes das colunas e um formato opcional
- `converters`: `dict` de {Hashable Callable}, optional. Aqui tem um peculiaridade: Se `converters` for definido, ele terá prioridade em vez do `dtype`.

Para passarmos essas estruturas para o método `read_csv()`, precisamos definilas:


In [5]:
DTYPES = {
    "ANO_BO": pd.StringDtype(),
    "ANO_FABRICACAO": pd.StringDtype(),
    "ANO_MODELO": pd.StringDtype(),
    "BO_AUTORIA": pd.StringDtype(),
    "CIDADE": pd.StringDtype(),
    "BAIRRO": pd.StringDtype(),
    "LOGRADOURO": pd.StringDtype(),
    "CIDADE_VEICULO": pd.StringDtype(),
    "CORCUTIS": None,
    "DELEGACIA_CIRCUNSCRICAO": pd.StringDtype(),
    "DELEGACIA_NOME": pd.StringDtype(),
    "DESCR_COR_VEICULO": pd.StringDtype(),
    "DESCR_MARCA_VEICULO": pd.StringDtype(),
    "DESCR_TIPO_VEICULO": pd.StringDtype(),
    "DESCRICAOLOCAL": pd.StringDtype(),
    "DESDOBRAMENTO": pd.StringDtype(),
    "ESPECIE": pd.StringDtype(),
    "ESTADOCIVIL": pd.StringDtype(),
    "EXAME": None,
    "FLAGRANTE": pd.StringDtype(),
    "GRAUINSTRUCAO": pd.StringDtype(),
    "IDADE": pd.Int64Dtype(),
    "MARCA_CELULAR": pd.StringDtype(),
    "NACIONALIDADE": pd.StringDtype(),
    "NATURALIDADE": pd.StringDtype(),
    "NATUREZAVINCULADA": pd.StringDtype(),
    "NUM_BO": pd.StringDtype(),
    "NUMERO": pd.StringDtype(),
    "NUMERO_BOLETIM": pd.StringDtype(),
    "NUMERO_BOLETIM_PRINCIPAL": pd.StringDtype(),
    "PARENTESCO": pd.StringDtype(),
    "PERIDOOCORRENCIA": pd.StringDtype(),
    "PLACA_VEICULO": pd.StringDtype(),
    "PROFISSAO": pd.StringDtype(),
    "QUANT_CELULAR": pd.Int64Dtype(),
    "RELACIONAMENTO": pd.StringDtype(),
    "RUBRICA": pd.StringDtype(),
    "SEXO": pd.StringDtype(),
    "SOLUCAO": pd.StringDtype(),
    "STATUS": pd.StringDtype(),
    "TIPOPESSOA": pd.StringDtype(),
    "TIPOVINCULO": pd.StringDtype(),
    "UF": pd.StringDtype(),
    "UF_VEICULO": pd.StringDtype(),
    "VITIMAFATAL": pd.StringDtype(),
}

DATE_TYPES = [
    "BO_EMITIDO",
    "BO_INICIADO",
    "DATACOMUNICACAO",
    "DATAELABORACAO",
    "DATANASCIMENTO",
    "DATAOCORRENCIA",
]

DATE_FORMAT = {
    "BO_EMITIDO": "%d/%m/%Y %H:%M:%S",
    "BO_INICIADO": "%d/%m/%Y %H:%M:%S",
    "DATACOMUNICACAO": "%d/%m/%Y",
    "DATAELABORACAO": "%d/%m/%Y %H:%M:%S",
    "DATANASCIMENTO": "%d/%m/%Y",
    "DATAOCORRENCIA": "%d/%m/%Y",
    "HORAOCORRENCIA": "%H:%M",
}

CONVERTERS = {
    "LATITUDE": clean_coordinate_value,
    "LONGITUDE": clean_coordinate_value,
}

TRANSFORM = [
    "ANO_BO",
    "ANO_FABRICACAO",
    "ANO_MODELO",
    "BO_AUTORIA",
    "CIDADE",
    "BAIRRO",
    "LOGRADOURO",
    "CIDADE_VEICULO",
    "DELEGACIA_CIRCUNSCRICAO",
    "DELEGACIA_NOME",
    "DESCR_COR_VEICULO",
    "DESCR_MARCA_VEICULO",
    "DESCR_TIPO_VEICULO",
    "DESCRICAOLOCAL",
    "DESDOBRAMENTO",
    "ESPECIE",
    "ESTADOCIVIL",
    "FLAGRANTE",
    "GRAUINSTRUCAO",
    "MARCA_CELULAR",
    "NACIONALIDADE",
    "NATURALIDADE",
    "NATUREZAVINCULADA",
    "NUM_BO",
    "NUMERO",
    "NUMERO_BOLETIM",
    "NUMERO_BOLETIM_PRINCIPAL",
    "PARENTESCO",
    "PERIDOOCORRENCIA",
    "PLACA_VEICULO",
    "PROFISSAO",
    "RELACIONAMENTO",
    "RUBRICA",
    "SEXO",
    "SOLUCAO",
    "STATUS",
    "TIPOPESSOA",
    "TIPOVINCULO",
    "UF",
    "UF_VEICULO",
    "VITIMAFATAL"
]


In [3]:
import os
import re
import shutil
import pandas as pd

In [4]:
"""
Tente limpar o dado e convertê-lo para float.
Caso não for possível a conversão e o dado não for None transforme em maiúsculo 


"""



def process_file(input_dir, output_dir):

    MONTHS = {
        "1": "Janeiro",
        "2": "Fevereiro",
        "3": "Março",
        "4": "Abril",
        "5": "Maio",
        "6": "Junho",
        "7": "Julho",
        "8": "Agosto",
        "9": "Setembro",
        "10": "Outubro",
        "11": "Novembro",
        "12": "Dezembro"
    }

    for filename in os.listdir(input_dir):
        # Verificar se o arquivo é um arquivo xls
        if filename.endswith(".xls"):
            # Pesquisar o padrão
            match = pattern.search(filename)
            # Se verdadeiro, capturar o dígito da string e utilizá-lo para obter o valor da chave do dict `meses`
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                # Novo nome do arquivo após substituição
                new_name = pattern.sub(f"{name_month}", filename)

                # Caminhos completos dos arquivos antigo e novo
                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path

In [6]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)


pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")

file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t',
                        encoding="UTF-16 LE",
                        dtype=DTYPES,
                        parse_dates=DATE_TYPES,
                        date_format=DATE_FORMAT,
                        converters=CONVERTERS)
            for file in file_path_generator]

df_temp = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)

## Limpeza dos dado

In [13]:
# https://pandas.pydata.org/docs/user_guide/text.html#string-methods 
for c in TRANSFORM:
    df_temp[c] = df_temp[c].str.upper()
    df_temp.columns.str.strip()

print(df_temp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

In [14]:
df_temp.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
39783,2023,580,580/2023,2023-06-07 19:18:15,2023-06-07 19:18:15,2023-06-04,23:33,A NOITE,2023-06-07,2023-06-07 19:18:15,DESCONHECIDA,NÃO,<NA>,AVENIDA FERNANDO FIGUEIREDO LINS,854,VILA CURUCA,S.PAULO,SP,-23.518841,-46.415013,VIA PÚBLICA,NaN,BO PARA INVESTIGAÇÃO,67º D.P. JARDIM ROBRU,67º D.P. JARDIM ROBRU,LOCALIZAÇÃO E/OU DEVOLUÇÃO,LOCALIZAÇÃO/APREENSÃO E ENTREGA DE VEÍCULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,FUS4267,SP,S.PAULO,CINZA,I/NISSAN VERSA 16SV FLEX,2014,2014,AUTOMOVEL,1,XIAOMI
71743,2023,310891,310891/2023,2023-02-27 16:56:41,2023-02-27 16:56:41,2023-02-23,22:04,A NOITE,2023-02-24,2023-02-27 16:56:41,DESCONHECIDA,NÃO,300556/2023 - 900020,RUA BILAC,489,CONJUNTO HABITACIONAL BRIGADEIRO FARIA L,S.PAULO,SP,-23.757420,-46.681663,OUTROS,NaN,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,85º D.P. JARDIM MIRNA,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,ENI3J18,SP,DIADEMA,PRETA,HYUNDAI/CRETA 16A SMART,2019,<NA>,AUTOMOVEL,1,MOTOROLA
4587,2023,75,75/2023,2023-01-07 01:32:14,2023-01-07 01:32:14,2023-01-07,00:30,DE MADRUGADA,2023-01-07,2023-01-07 01:32:14,DESCONHECIDA,NÃO,<NA>,RUA SHOZO SAKAI,239,VL CINTRA,MOGI DAS CRUZES,SP,-23.544601,-46.231222,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DEL.SEC.MOGI CRUZES PLANTÃO,02º D.P. MOGI DAS CRUZES,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - VEICULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,SAMSUNG
31363,2023,940,940/2023,2023-05-26 16:07:41,2023-05-26 16:07:41,2022-11-13,04:00,DE MADRUGADA,2022-11-13,2023-05-26 16:07:41,CONHECIDA,SIM,1685/2022 - 42428,<NA>,0,CHÁCARA GLÓRIA,ITANHAEM,SP,NaN,NaN,RESIDÊNCIA,NaN,BO PARA FLAGRANTE,DEL.SEC.ITANHAEM PLANTÃO,02º D.P. ITANHAEM,L 8069/90 - ECA,CAPUT CORROMPER OU FACILITAR A CORRUPÇÃO DE ME...,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,8,OUTROS
40875,2023,87906,87906/2023,2023-06-09 22:21:05,2023-06-09 22:21:05,2023-06-09,NaN,A NOITE,2023-06-09,2023-06-09 22:21:05,DESCONHECIDA,NÃO,<NA>,RODO ANEL MÁRIO COVAS,175,VILA MUNICIPAL,CARAPICUIBA,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA 2,01º D.P. CARAPICUIBA,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE


---

## Referencias